In [10]:
import yaml
import logging

In [11]:
# Initialize the logger once as the application starts up.
with open("../logging.yaml", 'rt') as f:
    config = yaml.safe_load(f.read())
    logging.config.dictConfig(config)
 
# Get an instance of the logger and use it to write a log!
# Note: Do this AFTER the config is loaded above or it won't use the config.
logger = logging.getLogger("py-clean-arch")
logger.info("Configured the logger!")

2023-03-08 09:59:35,929 - py-clean-arch - INFO - Configured the logger!


In [12]:
class NameSpace:
    """create and object from kwargs arguments"""
    ACCEPTED_VALUES = ("id_", "user", "location")

    def __init__(self, **data) -> None:
        for attr, value in data.items():
            if attr in self.ACCEPTED_VALUES:
                setattr(self,attr, value)

In [15]:
ns = NameSpace(id_=20,user="luis", location="bahamas", noacepted="novalue")
ns.__dict__

{'id_': 20, 'user': 'luis', 'location': 'bahamas'}

# About inheritance
scenario:
Imagine we have
a system for managing insurance, with a module in charge of applying policies
to different clients. We need to keep in memory a set of customers that are being processed at the time in order to apply those changes before further processing
or persistence. The basic operations we need are to store a new customer with its records as satellite data, apply a change to a policy, or edit some of the data, to name but a few. We also need to support a batch operation. That is, when something on the policy itself changes (the one this module is currently processing), we have to apply these changes overall to customers on the current transaction

In [17]:
from datetime import date

In [123]:
cl01 = {"cl01": {"fee":200.0, "date": date(2022,10,10)}}
cl02 = {"cl02": {"fee":300.0, "date": date(2022,10,10)}}
cl03 = {"cl03": {"fee":400.0, "date": date(2022,10,10)}}


cl01_update= {"cl01": {"date":date(2022,12,2)}}

In [124]:
class TransactionalPolicy:
    def __init__(self,policy, **extra_data) -> None:
        self._policies = {**policy, **extra_data}
    
    def update_policy(self, customer_id, policy_def):
        self._policies[customer_id].update(**policy_def)
    
    def __getitem__(self, idx):
        return self._policies[idx]

    def __len__(self):
        return len(self._policies)
    
    def __repr__(self) -> str:
        return str(self._policies.items())
    

In [125]:
policies = TransactionalPolicy({**cl01,**cl02})

In [132]:
policies["cl02"]

{'fee': 300.0, 'date': datetime.date(2022, 10, 10)}

# multiple inheritance

In [1]:
# python uses and algorithm called C3 linearization or MRO to fix class dependencies

class BaseModule:
    module_name = "base module"
    def __init__(self, module_name) -> None:
        self.module_name = module_name
    
    def __repr__(self) -> str:
        return f"module name: {self.module_name}, classname:{self.__class__.__name__}"

class BaseModuleOne(BaseModule):
    module_name="base module 01"

class BaseModuleTwo(BaseModule):
    module_name="base module 02"

class BaseModuleThree(BaseModule):
    module_name="base module 03"

class ConcreteModuleA(BaseModuleOne, BaseModuleTwo):
   """extends one and two"""

class ConcreteModuleB(BaseModuleTwo,BaseModuleThree):
    """extends two and four"""

In [2]:
ConcreteModuleA("test")

module name: test, classname:ConcreteModuleA

In [3]:
# to check the class resolution order
[cls.__name__ for cls in ConcreteModuleA.mro()]

['ConcreteModuleA', 'BaseModuleOne', 'BaseModuleTwo', 'BaseModule', 'object']

# mixins
A mixin is a base class that encapsulates some common behavior with the goal of reusing code.

In [4]:
class Tokenizer:
    def __init__(self, token) -> None:
        self.str_token = token
    def __iter__(self):
        yield from self.str_token.split("-")

In [5]:
[i for i in Tokenizer("asdf-asdfa-i99302")]

['asdf', 'asdfa', 'i99302']

In [9]:
# imagine we want to tokenize to upper case without changing behviour of class, use mixins

class UpperMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())
    

class UpperTokenizer(UpperMixin,Tokenizer):
    pass
    

In [13]:
ut = UpperTokenizer("dkdjd-wwow-dkdk")
[e for e in ut]
print([cls.__name__ for cls in UpperTokenizer.mro()])

['UpperTokenizer', 'UpperMixin', 'Tokenizer', 'object']


# in python all arguments are passed by value

In [17]:
inmutable = "hello"
mutable = list("hello")
print(inmutable)
print(mutable)

def mutable_function(param):
    param += " mutate param"
    return param

hello
['h', 'e', 'l', 'l', 'o']


In [18]:
print(mutable_function(inmutable))
print(inmutable)
print(mutable_function(mutable))
print(mutable)

hello mutate param
hello
['h', 'e', 'l', 'l', 'o', ' ', 'm', 'u', 't', 'a', 't', 'e', ' ', 'p', 'a', 'r', 'a', 'm']
['h', 'e', 'l', 'l', 'o', ' ', 'm', 'u', 't', 'a', 't', 'e', ' ', 'p', 'a', 'r', 'a', 'm']


In [19]:
# becarefull when passing parameters that are mutalbe ex. list

In [1]:
DEFAULT_TIMEOUT=30

def function(**kwargs): # wrong
    timeout = kwargs.get("timeout", DEFAULT_TIMEOUT) 
    pass

#Let Python do the unpacking and set the default argument at the signature:
def function(timeout=DEFAULT_TIMEOUT, **kwargs): # better 
    pass